In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from mlxtend.frequent_patterns import apriori, association_rules

# Sample data
data = pd.DataFrame({
    'client': [1, 2, 3, 4, 5],
    'product': [101, 102, 103, 101, 104],
    'income': [50000, 60000, 70000, 80000, 90000],
    'capital': [10000, 20000, 15000, 25000, 30000]
})

# Create a client-product interaction matrix
interaction_matrix = pd.crosstab(data['client'], data['product'])

# Calculate cosine similarity between products
product_similarity = cosine_similarity(interaction_matrix.T)
product_similarity_df = pd.DataFrame(product_similarity, index=interaction_matrix.columns, columns=interaction_matrix.columns)

# Market Basket Analysis
# Convert data into a basket format
basket = data.groupby(['client', 'product']).size().unstack(fill_value=0)

# Apply Apriori
frequent_itemsets = apriori(basket, min_support=0.1, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Recommend products function
def recommend_products(client_id, top_n=2, method='collaborative'):
    if method == 'collaborative':
        client_products = interaction_matrix.loc[client_id]
        similar_scores = product_similarity_df.dot(client_products).sort_values(ascending=False)
        recommended_products = similar_scores.index[similar_scores > 0].tolist()
        return recommended_products[:top_n]
    elif method == 'market_basket':
        client_products = set(interaction_matrix.loc[client_id][interaction_matrix.loc[client_id] > 0].index)
        recommendations = set()

        for _, rule in rules.iterrows():
            if client_products.issuperset(rule['antecedents']):
                recommendations.update(rule['consequents'])

        recommendations = list(recommendations - client_products)
        return recommendations[:top_n]

# Example recommendation
recommended_products_collab = recommend_products(client_id=1, top_n=2, method='collaborative')
recommended_products_mba = recommend_products(client_id=1, top_n=2, method='market_basket')

print("Collaborative Filtering Recommended products:", recommended_products_collab)
print("Market Basket Analysis Recommended products:", recommended_products_mba)


Collaborative Filtering Recommended products: [101]
Market Basket Analysis Recommended products: []


/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

# Sample data
data = pd.DataFrame({
    'client': [1, 2, 3, 4, 5],
    'product': [101, 102, 103, 101, 104],
    'income': [50000, 60000, 70000, 80000, 90000],
    'capital': [10000, 20000, 15000, 25000, 30000]
})

# Normalize income and capital
scaler = MinMaxScaler()
data[['income', 'capital']] = scaler.fit_transform(data[['income', 'capital']])

# Extract client features and fit K-means
client_features = data[['income', 'capital']].drop_duplicates()
kmeans = KMeans(n_clusters=2, random_state=42)
client_features['cluster'] = kmeans.fit_predict(client_features)

# Merge cluster information back to the original data
data = data.merge(client_features, on=['income', 'capital'])

# Function to recommend products
def recommend_products(client_id, top_n=2):
    client_cluster = data[data['client'] == client_id]['cluster'].values[0]
    cluster_data = data[data['cluster'] == client_cluster]
    popular_products = cluster_data['product'].value_counts().head(top_n).index.tolist()
    return popular_products

# Example recommendation
recommended_products = recommend_products(client_id=1, top_n=2)
print("Recommended products:", recommended_products)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Recommended products: [101, 102]


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [4]:
data

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,client,product,income,capital,cluster
0,1,101,0.00,0.00,1
1,2,102,0.25,0.50,1
2,3,103,0.50,0.25,1
3,4,101,0.75,0.75,0
4,5,104,1.00,1.00,0


In [6]:
import pandas as pd
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import numpy as np
# Sample data
data = pd.DataFrame({
    'client': [1, 2, 3, 4, 5],
    'product': [101, 102, 103, 101, 104],
    'income': [50000, 60000, 70000, 80000, 90000],
    'capital': [10000, 20000, 15000, 25000, 30000]
})

# Create a client-product interaction matrix
interaction_matrix = pd.crosstab(data['client'], data['product']).astype(float)
interaction_matrix.fillna(0, inplace=True)
# Ensure the interaction matrix is a numpy array of floats
interaction_matrix_np = interaction_matrix.to_numpy()
# Apply SVD
U, sigma, Vt = svds(interaction_matrix_np, k=2)
sigma = np.diag(sigma)

# Compute predicted ratings
predicted_ratings = np.dot(np.dot(U, sigma), Vt)
predicted_ratings_df = pd.DataFrame(predicted_ratings, columns=interaction_matrix.columns, index=interaction_matrix.index)

# Normalize income and capital
scaler = MinMaxScaler()
data[['income', 'capital']] = scaler.fit_transform(data[['income', 'capital']])

# Compute a feature score for each client-product pair
def compute_feature_score(client, product, income, capital):
    return income * 0.5 + capital * 0.5  # Example scoring formula

# Add feature scores to the predictions
for client in data['client'].unique():
    client_data = data[data['client'] == client]
    for product in data['product'].unique():
        feature_score = compute_feature_score(client, product, client_data['income'].values[0], client_data['capital'].values[0])
        if product in predicted_ratings_df.columns:
            predicted_ratings_df.loc[client, product] += feature_score

# Rank products based on the combined score
def rank_products(client_id, top_n=2):
    client_scores = predicted_ratings_df.loc[client_id].sort_values(ascending=False)
    ranked_products = client_scores.index.tolist()
    return ranked_products[:top_n]

# Example recommendation
ranked_products = rank_products(client_id=1, top_n=2)
print("Ranked products:", ranked_products)


Ranked products: [101, 102]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
data

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,client,product,income,capital
0,1,101,0.00,0.00
1,2,102,0.25,0.50
2,3,103,0.50,0.25
3,4,101,0.75,0.75
4,5,104,1.00,1.00


In [8]:
interaction_matrix

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


product,101,102,103,104
client,,,,
1,1.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0.0
4,1.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0


In [9]:
predicted_ratings_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


product,101,102,103,104
client,,,,
1,1.000,3.330669e-16,2.775558e-17,-1.110223e-16
2,0.375,1.025133e+00,5.551618e-01,-6.659019e-02
3,0.375,5.551618e-01,4.249256e-01,2.526285e-01
4,1.750,7.500000e-01,7.500000e-01,7.500000e-01
5,1.000,5.584098e-01,8.776285e-01,1.299942e+00
